## Import Necesary Libraries

In [60]:
import pandas as pd

## Load Datasets

In [61]:
df_omie_labelled = pd.read_csv('data\\df_omie_labelled.csv')
filtered_categories = pd.read_csv('data\\filtered_categories.csv')

## Merge Datasets

Add the technology type (Wind/Solar)

In [62]:
df = pd.merge(df_omie_labelled, filtered_categories, on='Codigo', how='left')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409526 entries, 0 to 2409525
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Codigo       object 
 1   Descripcion  object 
 2   fechaHora    object 
 3   PrecEuro     float64
 4   Energia      float64
 5   Categoria    object 
dtypes: float64(2), object(4)
memory usage: 110.3+ MB


## Change Data Types

Change fechaHora to Datetime data type

In [64]:
df["fechaHora"] = pd.to_datetime(df["fechaHora"])

## Create time-based features

Split datatime into hour, day of week and month

In [65]:
df['hour'] = df['fechaHora'].dt.hour
df['day_of_week'] = df['fechaHora'].dt.dayofweek
df['month'] = df['fechaHora'].dt.month

## Closer look to extra variables

In [66]:
unit_list = pd.read_csv('data\\unit_list.csv')
unit_list.head()

,Codigo,Descripción,Agente,Porcentaje_Propiedad,Tipo_Unidad,Zona/Frontera,Tecnología
0,AFIBGEC,ALMACENAMIENTO COMPRA,IBERDROLA ENERGÍA ESPAÑA S..A.,100.0,ALMACENAMIENTO,ZONA ESPAÑOLA,Almacenamiento Compra
1,AFIBHEC,COMPRA HIBRIDACION EOLICA BAT,IBERDROLA ENERGÍA ESPAÑA S..A.,100.0,ALMACENAMIENTO,ZONA ESPAÑOLA,Almacenamiento Compra
2,AFIBHFC,COMPRA HIBRIDACION FV BATERIA,IBERDROLA ENERGÍA ESPAÑA S..A.,100.0,ALMACENAMIENTO,ZONA ESPAÑOLA,Almacenamiento Compra
3,ACAVADB,BOMBAS DO CAVADO,EDP GEM PORTUGAL S.A.,100.0,BOMBEO,ZONA PORTUGUESA,Consumo Bombeo Mixto
4,ADOUROB,BOMBA DO DOURO,EDP GEM PORTUGAL S.A.,100.0,BOMBEO,ZONA PORTUGUESA,Consumo Bombeo Mixto


Tipo_Unidad not necessary, as all codes in the `df_omie_labelled` dataset refer to a "Generacion" type.

Zona/Frontera not necessary, as all codes in the `df_omie_labelled` dataset refer to a "ZONA ESPAÑOLA".

We will keep Tecnologia column and label encode it.

## Tecnologia Merging

In [67]:
df = pd.merge(df, unit_list, on='Codigo', how='left')
df = df.drop(columns=['Descripción', 'Agente', 'Porcentaje_Propiedad', 'Tipo_Unidad', 'Zona/Frontera'])

## Encoding Categorical Variables

In [68]:
df = pd.get_dummies(df, columns=["Categoria", "Tecnología"], drop_first=True)

## Final Dataset Overview

In [69]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409526 entries, 0 to 2409525
Data columns (total 13 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   Codigo                                     object        
 1   Descripcion                                object        
 2   fechaHora                                  datetime64[ns]
 3   PrecEuro                                   float64       
 4   Energia                                    float64       
 5   hour                                       int32         
 6   day_of_week                                int32         
 7   month                                      int32         
 8   Categoria_Solar                            bool          
 9   Tecnología_RE Mercado Solar Fotovoltáica   bool          
 10  Tecnología_RE Mercado Solar Térmica        bool          
 11  Tecnología_RE Tar. CUR Eólica              bool          
 12  

,Codigo,Descripcion,fechaHora,PrecEuro,Energia,hour,day_of_week,month,Categoria_Solar,Tecnología_RE Mercado Solar Fotovoltáica,Tecnología_RE Mercado Solar Térmica,Tecnología_RE Tar. CUR Eólica,Tecnología_RE Tar. CUR Solar Fotovoltáica
0,ADXRE01,PREAL1,2024-02-29,0.0,0.0,0,3,2,True,True,False,False,False
1,ADXVD03,PV ALARCOS,2024-02-29,0.0,0.0,0,3,2,True,True,False,False,False
2,ADXVD04,EOLICA AUDAX,2024-02-29,-0.1,2.7,0,3,2,False,False,False,False,False
3,AIRVD01,AIRVD01,2024-02-29,0.0,0.0,0,3,2,True,True,False,False,False
4,AIRVD02,AIRVD02,2024-02-29,0.0,0.0,0,3,2,True,True,False,False,False
